In [1]:
import tensorflow as tf
import datetime, os
from model import *
from data import *

#### Unpack videos and labels

In [2]:
unpack_video('data/larynx/train', video_folder='video', image_folder='image/glottis', target_size=(256, 256))
unpack_tif('data/larynx/train', tif_folder='tifs', label_folder='label/glottis', target_size=(256, 256))

#### Initialize the model (U-Net)

In [3]:
model = unet()
# model_checkpoint = ModelCheckpoint('unet_larynx_{epoch:02d}-{loss:.2f}.hdf5', monitor='loss', verbose=1, save_best_only=True)
model_checkpoint = ModelCheckpoint('unet_larynx.hdf5', monitor='val_loss', verbose=1, save_best_only=True)



#### Prepare training dataset

In [11]:
import splitfolders

splitfolders.ratio("data/larynx/train/image", output="data/larynx/train/image_split",
    seed=1234, ratio=(.8, .2), group_prefix=None, move=False) 
splitfolders.ratio("data/larynx/train/label", output="data/larynx/train/label_split",
    seed=1234, ratio=(.8, .2), group_prefix=None, move=False) 


training_dataset = training_dataset_generator(2,
                                              'data/larynx/train',
                                              'image_split/train/glottis',
                                              'label_split/train/glottis',
                                              save_to_dir=None)

val_dataset = training_dataset_generator(2,
                                              'data/larynx/train',
                                              'image_split/val/glottis',
                                              'label_split/val/glottis',
                                              save_to_dir=None)

Copying files: 55 files [00:00, 2109.93 files/s]
Copying files: 55 files [00:00, 1817.85 files/s]


In [12]:
# Set up TensorBoard callback
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    mode='min',
    restore_best_weights=True
)


# Train the model using the training dataset and TensorBoard callback
history = model.fit(training_dataset, 
          steps_per_epoch=10, 
          epochs=250, 
          callbacks=[model_checkpoint, tensorboard_callback, early_stopping],
          validation_data=val_dataset)

KeyError: 'glottis'

#### Testing the model

In [ ]:
# testing the model
test_dataset = test_dataset_generator("data/larynx/test", num_image=5)
results = model.predict(test_dataset, 3, verbose=1)
saveResult("data/larynx/test", results)


5/5 [==============================] - 1s 172ms/step


c:\Users\perseusf\anaconda3\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


55/55 [==============================] - 9s 172ms/step


c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\0_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\1_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\2_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\3_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\4_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/